# Semi-Automated Plate Definition Generation

- tags: #platedefinition #resourcemovement #plateadapter #hamiltonstar
- Last updated: 2026-01-26

## Prerequisites

- Machines used:
  - Hamilton STAR
- Non-PLR dependencies: None 


## Preview of Machine Behvaiour

<video width="640" controls autoplay loop>
  <source src="./assets/star_movement_plate_to_alpaqua_core/animation.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

## Protocol Mode

In [1]:
protocol_mode = "simulation" # "execution" or "simulation"

---
## Import Statements

### Non-PLR Dependencies

None

### Machine & Visualizer

In [2]:
%load_ext autoreload
%autoreload 2
       
import random 
import time

from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.resources.hamilton import STARLetDeck
from pylabrobot.visualizer.visualizer import Visualizer

if protocol_mode == "execution":

  from pylabrobot.liquid_handling.backends import STARBackend

  star = STARBackend()

elif protocol_mode == "simulation":

  from pylabrobot.liquid_handling.backends.hamilton.STAR_chatterbox import STARChatterboxBackend
    
  star = STARChatterboxBackend()

### Required Resources

In [3]:
from pylabrobot.resources import (
    hamilton_mfx_carrier_L5_base,
    hamilton_mfx_plateholder_DWP_metal_tapped,
    hamilton_mfx_plateholder_DWP_flat,
    Coordinate
)

#### Create Initial Plate Definition

In [4]:
from pylabrobot.resources.plate import Plate
from pylabrobot.resources.utils import create_ordered_items_2d
from pylabrobot.resources.well import (
    CrossSectionType,
    Well,
    WellBottomType,
)

def biorad_96_wellplate_200uL_Vb(name: str, with_lid: bool = False) -> Plate:
    """Bio-rad cat. no.: HSP9601 (50/package), HSP9601B (8*50/package).

    Bio-rad plate with 96-wells (V-bottoms).
    "The patented rigid 2-component design is specifically engineered to
      withstand the stresses of thermal cycling." -> excellent for automation!

    - Colour: white shell/clear well
    - alternative cat. no.:
      - red shell (HSP-9611)
      - yellow shell (HSP-9621)
      - blue shell (HSP-9631)
      - green shell (HSP-9641)
      - black shell (HSP-9661)
    - Material: Polypropylene
    - Cleanliness: "Certified to be free of DNase, RNase, and human genomic DNA"
    - Total volume/well = 200 uL
    - URL: https://www.bio-rad.com/en-uk/sku/HSP9601-hard-shell-96-well-pcr-plates-
      low-profile-thin-wall-skirted-white-clear?ID=HSP9601
    - technical drawing: ./techical_drawings/biorad_96_wellplate_200uL_Vb.png
    """
    well_diameter = 5.46  # mm
    return Plate(
        name=name,
        size_x=127.76,
        size_y=85.48,
        size_z=16.06,
        lid=None,
        model=biorad_96_wellplate_200uL_Vb.__name__,
        ordered_items=create_ordered_items_2d(
            Well,
            num_items_x=12,
            num_items_y=8,
            dx=11.65,
            dy=8.51,
            dz=1.1,
            item_dx=9.0,
            item_dy=9.0,
            size_x=well_diameter,
            size_y=well_diameter,
            size_z=14.4,
            bottom_type=WellBottomType.V,
            material_z_thickness=0.65,
            cross_section_type=CrossSectionType.CIRCLE,
            compute_volume_from_height=None,
            compute_height_from_volume=None,
        ),
    )

## Instantiate Frontend & Connect to Machine

In [5]:
deck = STARLetDeck()
lh = LiquidHandler(backend=star, deck=deck)

await lh.setup()

vis = Visualizer(resource=lh)
await vis.setup()

await star.disable_cover_control() # 😈

Websocket server started at http://127.0.0.1:2123
File server started at http://127.0.0.1:1339 . Open this URL in your browser.
C0CDid0001


## Configure Deck Layout

In [6]:
# Setup MFX Carrier with DWP PlateHolder that has a pedestal

plateholder_pedestal_0 = hamilton_mfx_plateholder_DWP_metal_tapped(
    name=f"plateholder_pedestal_0"
)

mfx_carrier_0 = hamilton_mfx_carrier_L5_base(
  name="mfx_carrier_0",
  modules={0: plateholder_pedestal_0}
)

deck.assign_child_resource(mfx_carrier_0, rails=1)

# Setup MFX Carrier with DWP PlateHolder that has a flat bottom

plateholder_flat_0 = hamilton_mfx_plateholder_DWP_flat(name=f"plateholder_flat_0")

mfx_carrier_1 = hamilton_mfx_carrier_L5_base(
  name="mfx_carrier_1",
  modules={0: plateholder_flat_0}
)

pcr_plate_0 = biorad_96_wellplate_200uL_Vb(name="pcr_plate_0")

mfx_carrier_1[0] = pcr_plate_0 

deck.assign_child_resource(mfx_carrier_1, rails=8)

---
## Execution

### Store Coordinates to Probe on Flat

In [7]:
def generate_channel_wells(plate, channels_used):
    """
    Generate a list of wells for multi-channel pipetting with evenly spaced rows.
    
    Returns:
        List of well names (e.g., ["A1", "C1", "E1", "H1", "A2", "C2", ...])
    """
    num_rows, num_cols = plate.num_items_y, plate.num_items_x
    
    # Generate evenly spaced row indices
    row_indices = [round(i * (num_rows - 1) / (channels_used - 1)) for i in range(channels_used)]
    
    # Generate well names: all columns, with selected rows per column
    return [f"{chr(ord('A') + row)}{col}" 
            for col in range(1, num_cols + 1) 
            for row in row_indices]

In [8]:
n_channels = 3
wells_to_probe = generate_channel_wells(pcr_plate_0, channels_used=n_channels)

positions_to_probe_flat = [
    well.get_absolute_location("c", "c", "top")
    for well in pcr_plate_0.children
        if well.get_identifier() in wells_to_probe
]

### Move Plate to PlateHolder with Pedestal

In [9]:
teaching_tip_rack = lh.deck.get_resource("teaching_tip_rack")
await lh.pick_up_tips(teaching_tip_rack["A1:C1"], use_channels=[0,1,2,3,4,5][:n_channels])

await star.pick_up_core_gripper_tools(front_channel=7)

C0TTid0002tt01tf1tl0519tv03600tg2tu0
C0TPid0003xp07854 07854 07854 00000&yp5286 5196 5106 0000&tm1 1 1 0&tt01tp1830tz1750th2450td0
C0ZTid0004xs07975xd0ya1250yb1070pa07pb08tp2350tz2250th2800tt14


In [10]:
move_target = mfx_carrier_0[0]

if protocol_mode == "simulation":
  time.sleep(2)
    
await lh.move_plate(
  plate=pcr_plate_0,
  to=move_target,
  use_arm="core",
  pickup_distance_from_top=6,
  core_grip_strength=40,
  return_core_gripper=False,
)

if protocol_mode == "execution":
  # "smart" command, will ask operator for input if it cannot find plate in
  # move_target location place into condition for simulation mode

  # (1) check transfer success, (2) push plate flush
  await star.core_check_resource_exists_at_location_center(
    location=pcr_plate_0.get_absolute_location(),
    resource=pcr_plate_0,
    gripper_y_margin=9,
    enable_recovery=True,
    audio_feedback=False,
  )

print(star.core_parked)

if protocol_mode == "simulation":
  time.sleep(2)

C0ZPid0005xs03254xd0yj1142yv0050zj1881zy0500yo0885yg0825yw40th2800te2800
C0ZRid0006xs01679xd0yj1147zj1929zi000zy0500yo0885th2800te2800
False


### Store Coordinates to Probe on Pedestal

In [11]:
positions_to_probe_pedestal = [
    well.get_absolute_location("c", "c", "top")
    for well in pcr_plate_0.children
        if well.get_identifier() in wells_to_probe
]

### Probe True PlateHolder Origin

In [12]:
plate_holder_flat_child_coord = mfx_carrier_1[0].get_absolute_location()+mfx_carrier_1[0].child_location

test_location_w_offset = plate_holder_flat_child_coord + Coordinate(x=3, y=3, z=0)

In [17]:
channel_idx = 0

# Prepare Z position of X-Y probing
await star.move_all_channels_in_z_safety()
await star.prepare_for_manual_channel_operation(channel=channel_idx)

await star.move_channel_x(x=test_location_w_offset.x, channel=channel_idx)
await star.move_channel_y(y= test_location_w_offset.y, channel=channel_idx)

probed_z_position = await star.clld_probe_z_height_using_channel(
        channel_idx=channel_idx,
        channel_speed=6.0, start_pos_search=test_location_w_offset.z+20,
        detection_edge=5, post_detection_dist=5.0,
        move_channels_to_safe_pos_after=False
    ) if protocol_mode == "execution" else plate_holder_flat_child_coord.z
await star.move_channel_z(z=probed_z_position+1.5, channel=channel_idx)

# X-Y Probing
offset_n_replicates = 3

front_pos_list = []
for x in range(offset_n_replicates):
    probed_front_position = await star.clld_probe_y_position_using_channel(
        channel_idx=channel_idx,
        probing_direction="forward",
        channel_speed=3,
        post_detection_dist=3.0,
        end_pos_search=test_location_w_offset.y-5
    ) if protocol_mode == "execution" else plate_holder_flat_child_coord.y
    front_pos_list.append(probed_front_position)

left_pos_list =[]
for x in range(offset_n_replicates):
    probed_left_position = await star.clld_probe_x_position_using_channel(
        channel_idx=channel_idx,
        probing_direction="left",
        post_detection_dist=1.0,
        end_pos_search=test_location_w_offset.x-5
    ) if protocol_mode == "execution" else plate_holder_flat_child_coord.x
    left_pos_list.append(probed_left_position)

await star.move_all_channels_in_z_safety()

true_plate_holder_flat_child_coord = Coordinate(
    x=round(sum(left_pos_list)/len(left_pos_list), 1),
    y=round(sum(front_pos_list)/len(front_pos_list), 1),
    z=probed_z_position,
)

C0ZAid0023
C0JPid0024pn01
C0JXid0025xs02645
moving channel 0 to y: 74.5
C0KZid0026pn01zj1795
C0ZAid0027


### Move Plate back onto tapped PlateHolder

In [18]:
move_target = mfx_carrier_1[0]

await lh.move_plate(
  plate=pcr_plate_0,
  to=move_target,
  use_arm="core",
  pickup_distance_from_top=6,
  core_grip_strength=40,
  return_core_gripper=False,
)

if protocol_mode == "execution":

  await star.core_check_resource_exists_at_location_center(
    location=pcr_plate_0.get_absolute_location(),
    resource=pcr_plate_0,
    gripper_y_margin=9,
    enable_recovery=True,
    audio_feedback=False,
  )

C0ZPid0028xs01679xd0yj1147yv0050zj1929zy0500yo0885yg0825yw40th2800te2800
C0ZRid0029xs03254xd0yj1142zj1881zi000zy0500yo0885th2800te2800


### Interactive X-Y Verification of Bottom-Left Well

In [19]:
test_location = pcr_plate_0.column(0)[-1].get_absolute_location(
    x="center",y="center",z="top"
)

await lh.backend.move_all_channels_in_z_safety()

await lh.backend.prepare_for_manual_channel_operation(channel=channel_idx)

# Position channel with teaching needle above bottom-left well
await lh.backend.move_channel_x(x=test_location.x, channel=channel_idx)
await lh.backend.move_channel_y(y=test_location.y, channel=channel_idx)
await lh.backend.move_channel_z(z=test_location.z+5, channel=channel_idx)


C0ZAid0030
C0JPid0031pn01
C0JXid0032xs02759
moving channel 0 to y: 82.74
C0KZid0033pn01zj1985


In [28]:
import tkinter as tk
from tkinter import ttk
import asyncio
import threading

def create_gui(lh, channel_idx, plateholder_child_location):
    """
    Create a GUI to control liquid handler position
    
    Args:
        lh: Liquid handler instance with backend
        channel_idx: Channel index to control
        plateholder_child_location: Reference location to calculate relative position from
    """
    
    # Create main window
    root = tk.Tk()
    root.title("Simple Counter GUI")
    root.geometry("1050x350")
    
    # Store reference location
    ref_pos = {
        'X': plateholder_child_location.x,
        'Y': plateholder_child_location.y,
        'Z': plateholder_child_location.z
    }
    
    # Track positions locally (fallback for simulation)
    current_positions = {
        'X': plateholder_child_location.x,
        'Y': plateholder_child_location.y,
        'Z': plateholder_child_location.z
    }
    
    # Labels to update
    float_labels1 = {}  # Relative to PlateHolder Child Location
    float_labels2 = {}  # Relative to Deck (absolute)
    resolution = tk.DoubleVar(value=0.1)  # Default to 0.1 mm
    
    # Define colors for each section
    colors = {'X': '#658F11', 'Y': '#B63E4B', 'Z': '#3E7DD7'}
    
    def run_async(coro):
        """Helper to run async coroutine and schedule GUI updates"""
        def run_in_thread():
            try:
                # Get or create event loop for this thread
                try:
                    loop = asyncio.get_event_loop()
                except RuntimeError:
                    loop = asyncio.new_event_loop()
                    asyncio.set_event_loop(loop)
                
                # Run the coroutine
                loop.run_until_complete(coro)
            except Exception as e:
                print(f"Error in async operation: {e}")
        
        # Run in background thread to not block tkinter
        thread = threading.Thread(target=run_in_thread, daemon=True)
        thread.start()
    
    async def get_current_positions():
        """Get current channel positions from backend"""
        try:
            x_pos = await lh.backend.request_x_pos_channel_n(pipetting_channel_index=channel_idx)
            y_pos = await lh.backend.request_y_pos_channel_n(pipetting_channel_index=channel_idx)
            z_pos = await lh.backend.request_z_pos_channel_n(pipetting_channel_index=channel_idx)
            
            # If any position is None, use locally tracked positions (simulation fallback)
            if x_pos is None or y_pos is None or z_pos is None:
                return current_positions.copy()
            
            # Update locally tracked positions
            current_positions['X'] = x_pos
            current_positions['Y'] = y_pos
            current_positions['Z'] = z_pos
            
            return {'X': x_pos, 'Y': y_pos, 'Z': z_pos}
        except Exception as e:
            print(f"Error getting positions: {e}")
            # Return locally tracked positions as fallback
            return current_positions.copy()
    
    def update_displays():
        """Update all position displays from current backend positions"""
        async def async_update():
            try:
                pos = await get_current_positions()
                
                # Schedule GUI update on main thread
                def gui_update():
                    for axis in ['X', 'Y', 'Z']:
                        absolute_pos = pos[axis]
                        relative_pos = absolute_pos - ref_pos[axis]
                        
                        # Update relative position (to plateholder child location)
                        float_labels1[axis].config(text=f"{relative_pos:.1f}")
                        
                        # Update absolute position (to deck)
                        float_labels2[axis].config(text=f"{absolute_pos:.1f}")
                
                root.after(0, gui_update)
                
            except Exception as e:
                print(f"Error updating displays: {e}")
        
        run_async(async_update())
    
    def update_counter(section, change):
        """Update the counter for a given section based on resolution"""
        step = resolution.get()
        
        async def async_move():
            try:
                # Get current position
                pos = await get_current_positions()
                
                # Calculate new position
                new_pos = pos[section] + (change * step)
                
                # Update locally tracked position
                current_positions[section] = new_pos
                
                # Send movement command - use 'channel' parameter
                if section == 'X':
                    await lh.backend.move_channel_x(x=new_pos, channel=channel_idx)
                elif section == 'Y':
                    await lh.backend.move_channel_y(y=new_pos, channel=channel_idx)
                elif section == 'Z':
                    await lh.backend.move_channel_z(z=new_pos, channel=channel_idx)
                
                # Update displays after movement
                await asyncio.sleep(0.1)  # Small delay for position to settle
                pos_updated = await get_current_positions()
                
                # Schedule GUI update on main thread
                def gui_update():
                    absolute_pos = pos_updated[section]
                    relative_pos = absolute_pos - ref_pos[section]
                    float_labels1[section].config(text=f"{relative_pos:.1f}")
                    float_labels2[section].config(text=f"{absolute_pos:.1f}")
                
                root.after(0, gui_update)
                
            except Exception as e:
                print(f"Error moving {section} axis: {e}")
        
        run_async(async_move())
    
    # Create a frame for the description labels
    desc_frame = tk.Frame(root)
    desc_frame.grid(row=0, column=0, padx=(10, 5), pady=20, sticky="e")
    
    # Add description labels
    desc_label1 = tk.Label(desc_frame, text="Relative to PlateHolder\nChild Location:", 
                           font=('Arial', 11, 'bold'), justify='right')
    desc_label1.pack(pady=(40, 25))
    
    desc_label2 = tk.Label(desc_frame, text="Relative to Deck:", 
                           font=('Arial', 11, 'bold'), justify='right')
    desc_label2.pack()
    
    # Create sections for X, Y, and Z
    for i, section in enumerate(['X', 'Y', 'Z']):
        # Outer container frame
        container = tk.Frame(root)
        container.grid(row=0, column=i+1, padx=10, pady=20, sticky="nsew")
        
        # Colored header label
        header = tk.Label(container, text=f"{section}", 
                         font=('Arial', 12, 'bold'), bg=colors[section],
                         fg='white', pady=5)
        header.pack(fill='x')
        
        # Content frame with white/default background
        frame = tk.Frame(container, borderwidth=2, relief='solid', padx=10, pady=10)
        frame.pack(fill='both', expand=True)
        
        # Minus button
        minus_btn = ttk.Button(frame, text="-", width=8,
                               command=lambda s=section: update_counter(s, -1))
        minus_btn.grid(row=0, column=0, padx=5, pady=5)
        
        # Plus button
        plus_btn = ttk.Button(frame, text="+", width=8,
                              command=lambda s=section: update_counter(s, 1))
        plus_btn.grid(row=0, column=1, padx=5, pady=5)
        
        # First float label - Relative to PlateHolder Child Location
        float_label1 = tk.Label(frame, text="0.0", font=('Arial', 12, 'bold'))
        float_label1.grid(row=1, column=0, columnspan=2, pady=(15, 5))
        float_labels1[section] = float_label1
        
        # Second float label - Relative to Deck (absolute position)
        float_label2 = tk.Label(frame, text=f"{ref_pos[section]:.1f}", 
                               font=('Arial', 12, 'bold'))
        float_label2.grid(row=2, column=0, columnspan=2, pady=5)
        float_labels2[section] = float_label2
    
    # Resolution section
    resolution_frame = ttk.LabelFrame(root, text="Resolution", padding=15, labelanchor='n')
    resolution_frame.grid(row=1, column=1, columnspan=3, padx=10, pady=(10, 20), sticky="ew")
    
    # Configure resolution label font
    style = ttk.Style()
    style.configure('Bold.TLabelframe.Label', font=('Arial', 12, 'bold'))
    resolution_frame.configure(style='Bold.TLabelframe')
    
    # Create inner frame to center the radio buttons
    radio_container = tk.Frame(resolution_frame)
    radio_container.pack(expand=True)
    
    # Create radio buttons for resolution
    resolutions = [
        ("0.1 mm", 0.1),
        ("1 mm", 1.0),
        ("10 mm", 10.0)
    ]
    
    for i, (text, value) in enumerate(resolutions):
        rb = tk.Radiobutton(radio_container, text=text, variable=resolution, 
                           value=value, font=('Arial', 11, 'bold'))
        rb.pack(side=tk.LEFT, padx=20)
    
    # Configure grid weights for responsive layout
    for i in range(1, 4):
        root.columnconfigure(i, weight=1)
    
    # Initialize displays with current positions (delayed to let GUI render)
    root.after(100, update_displays)
    
    # Start the GUI event loop
    root.mainloop()

create_gui(lh, channel_idx, true_plate_holder_flat_child_coord)

C0RXid0074
Error getting positions: 'NoneType' object is not subscriptable
C0RXid0075
Error getting positions: 'NoneType' object is not subscriptable
C0JXid0076xs02616
C0RXid0077
Error getting positions: 'NoneType' object is not subscriptable
C0RXid0078
Error getting positions: 'NoneType' object is not subscriptable
C0JXid0079xs02617
C0RXid0080
Error getting positions: 'NoneType' object is not subscriptable
C0RXid0081
Error getting positions: 'NoneType' object is not subscriptable
C0JXid0082xs02618
C0RXid0083
Error getting positions: 'NoneType' object is not subscriptable
C0RXid0084
Error getting positions: 'NoneType' object is not subscriptable
C0JXid0085xs02619
C0RXid0086
Error getting positions: 'NoneType' object is not subscriptable
C0RXid0087
Error getting positions: 'NoneType' object is not subscriptable
C0KZid0088pn01zj1790
C0RXid0089
Error getting positions: 'NoneType' object is not subscriptable


### Z Probing of Wells (on Flat PlateHolder)

### Move Plate to PlateHolder with Pedestal

### Z Probing of Flat PlateHolder

### Z Probing of Wells (on Pedestal)

### Update Plate Definition